#### Testing combinations with starts and stops as indices

This is intended to be used with oamap version ( I know we don't intend to finally use it, but let's go with it for now).

##### Idea for calculation

Using the kernel structure used in test_combinations, we can  generate combinations arrays for every event. 

For that, each event is assigned a block ( i.e. each block in the grid serves as an event), and the combinations is calculated for each block using the `starts[block_index]` and event length.
This is appended to `left` and `right` arrays. 

#### Sequential Version

In [1]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [2]:
# The generation step. Let's form a random integer array, from which we will form starts 
# and stops arrays
base_len = 32
base_arr = numpy.random.randint(6, size=base_len)

In [3]:
cumul_arr = numpy.zeros(base_len+1)
cumul_arr[0] = 0
cumul_arr[1:] = numpy.cumsum(base_arr)

In [4]:
start = cumul_arr[:-2]
stop = cumul_arr[1:-1]

In [5]:
start.shape

(31,)

In [6]:
stop[:5]

array([ 0.,  2.,  5.,  5.,  7.])

In [14]:
lengths = stop-start
pairs_lengths = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths*lengths)
lengths_arr = numpy.array([base_len-1]).astype(numpy.int32)
lengths = lengths.astype(numpy.int32)
start = start.astype(numpy.int32)
stop = stop.astype(numpy.int32)

In [8]:
pairs_lengths.shape

(32,)

In [9]:
lengths.shape

(31,)

In [11]:
left = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)

In [16]:
for i in range(base_len-1):
    pairs_i = pairs_lengths[i]
    for j in range(start[i], stop[i]):
        for k in range(start[i], stop[i]):
            left[pairs_i] = j
            right[pairs_i] = k
            pairs_i +=1

In [17]:
for i in range(6):
    print("Event: {} \n Left:{} \n Right:{} \n".format(i, left[pairs_lengths[i]:pairs_lengths[i+1]], right[pairs_lengths[i]:pairs_lengths[i+1]]))

Event: 0 
 Left:[] 
 Right:[] 

Event: 1 
 Left:[0 0 1 1] 
 Right:[0 1 0 1] 

Event: 2 
 Left:[2 2 2 3 3 3 4 4 4] 
 Right:[2 3 4 2 3 4 2 3 4] 

Event: 3 
 Left:[] 
 Right:[] 

Event: 4 
 Left:[5 5 6 6] 
 Right:[5 6 5 6] 

Event: 5 
 Left:[7 7 8 8] 
 Right:[7 8 7 8] 

